# Digital House - Trabajo Práctico Nº1
## Desafío 1. Análisis exploratorio de un dataset de precios de propiedades
Grupo Nª2
Alumnos:
* Alberto Paparelli
* André Song
* Marianela Riz
* Gloria Caravajal
* Franco Emanuel Rodriguez
* Iván Axel Schweizer

Objetivos:
* Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
* Realizar un análisis descriptivo de las principales variables.
* Crear nuevas columnas a partir de las características dadas que puedan tener valor
predictivo.

In [3]:
# Lo primero que hacemos es importar todas las librerias necesarias.
import pandas as pd

In [4]:
# Abrimos el dataset y hacemos un chequeo rápido para saber cantidad de registros y miramos la composición del mismo.
filename = "../dataset/properati.csv"
data = pd.read_csv(filename, on_bad_lines='skip')
display(data.sample(n=6)) 
print(f"(Cantidad de Filas, Cantidad de columnas) -> {data.shape}")

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
13198,13198,sell,apartment,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,NaN,NaN,...,27.0,1318.665000,33490.185185,2.0,NaN,NaN,http://www.properati.com.ar/16ocp_venta_depart...,LA NAZARENAModerno complejo de casas y departa...,"Departamento 40m² con Cochera, Bs.As. G.B.A. Z...",https://thumbs4.properati.com/2/3LBruGWFNPGLN0...
65167,65167,sell,house,Pilar,|Argentina|Bs.As. G.B.A. Zona Norte|Pilar|,Argentina,Bs.As. G.B.A. Zona Norte,3429979.0,NaN,NaN,...,600.0,NaN,1166.666667,NaN,NaN,NaN,http://www.properati.com.ar/1abw2_venta_casa_p...,"Km 46 de Panamericana Ramal Pilar, Santa Paula...",Casa - Pilar,NaN
27318,27318,sell,house,Longchamps,|Argentina|Bs.As. G.B.A. Zona Sur|Almirante Br...,Argentina,Bs.As. G.B.A. Zona Sur,3431259.0,"-34.862975,-58.392849",-34.862975,...,20.0,NaN,4500.000000,NaN,3.0,NaN,http://www.properati.com.ar/17t8q_venta_casa_l...,Corredor Responsable: Marcela Lucero - CMCPDJM...,CASA 3 AMB CON GALPÓN 2 COCHERAS,https://thumbs4.properati.com/2/WXq7DvYNjzJpNO...
24751,24751,sell,apartment,Recoleta,|Argentina|Capital Federal|Recoleta|,Argentina,Capital Federal,3429595.0,"-34.5922662367,-58.3853687613",-34.592266,...,189.0,3486.772487,3486.772487,NaN,6.0,NaN,http://www.properati.com.ar/17p19_venta_depart...,Espectacular departamento en una de las mejore...,Departamento en venta en inmejorable ubicación...,https://thumbs4.properati.com/8/jMgqoybkj2sp-t...
66790,66790,sell,house,Haras Santa Maria,|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|Ha...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,260.0,1067.251462,1403.846154,NaN,NaN,NaN,http://www.properati.com.ar/1aed4_venta_casa_h...,Casa de excelente diseño y calidad de terminac...,Excelente diseño y calidad de contrucción,https://thumbs4.properati.com/1/NBV94vL3icMH3d...
84934,84934,sell,apartment,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,3436397.0,NaN,NaN,...,48.0,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1b2cp_venta_depart...,BBU2970 AP192474,Torre Lumiere,https://thumbs4.properati.com/0/DCMr6Vb7UJJ385...


(Cantidad de Filas, Cantidad de columnas) -> (121220, 26)


# Columnas
* operation         -> Tipo de operación (Rent/Sell)
* property_type     -> Tipo de propiedad

COMPLETAR!!!!!

A continuación analizamos cada una de las columnas.

## Columna operations
Columna operation: revisamos que opciones tiene, o si se repite lo mismo.

In [5]:
# Columna operation: revisamos que opciones tiene, o si se repite lo mismo.
print(f'Cantidad de registros con NaN: {data.operation.isnull().sum()}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['operation']).size().reset_index(name='count'))


Cantidad de registros con NaN: 0
Contamos cantidad de registros unicos:
  operation   count
0      sell  121220


Detalle de análisis de columnas:
* `operation` -> Vemos que no tiene NaNs y solo tiene la opción `sell`. Por este motivo la vamos a eliminar.

In [6]:
# Borrado de columna operation. Utilizamos inplace=True para que no imprima el resultado.
data.drop(['operation'], axis=1, inplace=True)